In [1]:
import pandas as pd
import datetime

In [2]:
historical_data = pd.read_csv("csv_import/historical-data-011019.csv")

In [3]:
print(historical_data.head())

         Date         Type Confirmation Code  Start Date  Nights  \
0  01/10/2019       Payout               NaN         NaN     NaN   
1  01/10/2019  Reservation        HM8AX22DEN  01/09/2019     4.0   
2  01/10/2019  Reservation        HM5X9ETCT9  01/09/2019     3.0   
3  01/10/2019  Reservation        HMFCTS4CMZ  01/09/2019     2.0   
4  01/10/2019       Payout               NaN         NaN     NaN   

              Guest                                            Listing  \
0               NaN                                                NaN   
1     Naomi Gabizon   Earth Charter - Private Room in a Warm Community   
2  Aki Koivistoinen  Private Suite w/ Bath, Artsy Hotel/Co-op: Suns...   
3        Kyle Henry  Rose Garden - a Magical Bed+Bath in Warm Commu...   
4               NaN                                                NaN   

                 Details  Reference Currency  Amount  Paid Out  Host Fee  \
0  Transfer to *****6485        NaN      USD     NaN    823.62       N

In [4]:
# Month and Year columns added for accounting purposes
historical_data['Month'] = historical_data.Date.apply(lambda dateCol: datetime.datetime.strptime(dateCol,'%m/%d/%Y').month)
historical_data['Year'] = historical_data.Date.apply(lambda dateCol: datetime.datetime.strptime(dateCol,'%m/%d/%Y').year)
data = historical_data[historical_data.Year == 2018]

print(data.head())
print(data.dtypes)
print(data.Type.unique())

          Date         Type Confirmation Code  Start Date  Nights  \
86  12/31/2018       Payout               NaN         NaN     NaN   
87  12/31/2018  Reservation        HM2SEYSSAD  12/30/2018     3.0   
88  12/31/2018  Reservation        HM49ZEW3KT  12/30/2018     4.0   
89  12/31/2018  Reservation        HMFF5JXEHE  12/30/2018     3.0   
90  12/30/2018  Reservation        HM8TQXP2ST  12/29/2018     4.0   

                  Guest                                            Listing  \
86                  NaN                                                NaN   
87  Kathrin Bimesdörfer  Rose Garden - a Magical Bed+Bath in Warm Commu...   
88           Amy Torres  Private Suite w/ Bath, Artsy Hotel/Co-op: Suns...   
89       Andrea Godinez     Private Room in Historic B&B - Playground Room   
90            Hanna Kim    Golden Gate - Private Bedroom in Warm Community   

                  Details  Reference Currency  Amount  Paid Out  Host Fee  \
86  Transfer to *****4871        NaN   

In [5]:
# Payouts by month
payouts = data[data.Type == 'Payout']
print(payouts.groupby('Month')['Paid Out'].sum())

Month
1     16787.60
2     22148.36
3     30775.08
4     28832.03
5     32383.36
6     36164.04
7     43119.55
8     57299.25
9     63498.36
10    54980.71
11    34935.43
12    42231.70
Name: Paid Out, dtype: float64


In [6]:
# Gross hotel revenue by room/listing
payouts = data[data.Type.isin(['Reservation','Adjustment'])].groupby(['Month','Listing']).Amount.sum()
print(payouts.head())
payouts.to_csv('csv_export/gross_data_2018.csv')

Month  Listing                                          
1      Air Bunk, Peacock Room: Cozy Bed in Shared Room       197.88
       Earth Bunk, Peacock Room: Cozy Bed in Shared Room     948.66
       Fire Bunk, Peacock Room: Cozy Bed in Shared Room      230.86
       Friends - Private Bedroom for 3 in Cozy Community    1874.04
       Gigi's Room - a Private Suite in a Warm Community    2172.12
Name: Amount, dtype: float64


In [7]:
# Total unassigned resolution adjustments by month
adjustments = data[data.Type.isin(['Resolution Adjustment','Resolution Payout'])].groupby('Month').Amount.sum()
print(adjustments)

Month
3     -50.00
4    -100.00
5     -25.00
6    -181.64
7    -117.59
8    -638.92
9     210.00
10    171.05
12    -80.63
Name: Amount, dtype: float64


In [8]:
# Amounts from payouts and adjustments
adjustments = adjustments.reset_index()
print(adjustments.head())
payouts = payouts.reset_index()
print(payouts.head())

   Month  Amount
0      3  -50.00
1      4 -100.00
2      5  -25.00
3      6 -181.64
4      7 -117.59
   Month                                            Listing   Amount
0      1    Air Bunk, Peacock Room: Cozy Bed in Shared Room   197.88
1      1  Earth Bunk, Peacock Room: Cozy Bed in Shared Room   948.66
2      1   Fire Bunk, Peacock Room: Cozy Bed in Shared Room   230.86
3      1  Friends - Private Bedroom for 3 in Cozy Community  1874.04
4      1  Gigi's Room - a Private Suite in a Warm Community  2172.12


In [9]:
# Distribute unassigned resolution adjustments by month by rooms listed that month
rooms_per_month = payouts.groupby('Month').Listing.count().reset_index()
print(rooms_per_month)

monthly_distribution = pd.merge(adjustments, rooms_per_month, on='Month')
monthly_distribution['Amount Per Listing'] = (monthly_distribution.Amount / monthly_distribution.Listing).round(2)
print(monthly_distribution)

monthly_distribution = monthly_distribution.drop(['Amount', 'Listing'], axis=1)
print(monthly_distribution)

    Month  Listing
0       1       13
1       2       17
2       3       17
3       4       16
4       5       19
5       6       20
6       7       21
7       8       24
8       9       25
9      10       27
10     11       26
11     12       29
   Month  Amount  Listing  Amount Per Listing
0      3  -50.00       17               -2.94
1      4 -100.00       16               -6.25
2      5  -25.00       19               -1.32
3      6 -181.64       20               -9.08
4      7 -117.59       21               -5.60
5      8 -638.92       24              -26.62
6      9  210.00       25                8.40
7     10  171.05       27                6.34
8     12  -80.63       29               -2.78
   Month  Amount Per Listing
0      3               -2.94
1      4               -6.25
2      5               -1.32
3      6               -9.08
4      7               -5.60
5      8              -26.62
6      9                8.40
7     10                6.34
8     12               -2.78


In [10]:
# Create new data frame to hold adjustments for listings
adjusted_payouts = pd.merge(payouts, monthly_distribution, how='left', on=['Month','Month'])
adjusted_payouts = adjusted_payouts.fillna(0)
adjusted_payouts['Adjusted Amount'] = adjusted_payouts.Amount + adjusted_payouts['Amount Per Listing']

adjusted_payouts = adjusted_payouts.drop('Amount Per Listing', axis=1)
print(adjusted_payouts.head())

# Round amounts to 2 decimal points for dollars and cents (0.00)
adjusted_payouts = adjusted_payouts.round(2)

# Save adjusted payouts to new csv file
adjusted_payouts.to_csv('csv_export/adjusted_payouts_2018.csv')

   Month                                            Listing   Amount  \
0      1    Air Bunk, Peacock Room: Cozy Bed in Shared Room   197.88   
1      1  Earth Bunk, Peacock Room: Cozy Bed in Shared Room   948.66   
2      1   Fire Bunk, Peacock Room: Cozy Bed in Shared Room   230.86   
3      1  Friends - Private Bedroom for 3 in Cozy Community  1874.04   
4      1  Gigi's Room - a Private Suite in a Warm Community  2172.12   

   Adjusted Amount  
0           197.88  
1           948.66  
2           230.86  
3          1874.04  
4          2172.12  
